# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation. To run this example Julia's `PyCall` package
needs to be able to find an installation of `pymatgen`.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single k-point and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=(1, 1, 1))

PlaneWaveBasis discretization:
    Ecut                 : 5.0 Ha
    fft_size             : (32, 32, 32), 32768 total points
    kgrid type           : Monkhorst-Pack
    kgrid                : [1, 1, 1]
    num. irred. kpoints  : 1

    Discretized Model(lda_xc_teter93, 3D):
        lattice (in Bohr)    : [0         , 10.2631   , 10.2631   ]
                               [10.2631   , 0         , 10.2631   ]
                               [10.2631   , 10.2631   , 0         ]
        unit cell volume     : 2162.1 Bohr³
    
        atoms                : Si₁₆
        atom potentials      : ElementPsp(Si, psp="hgh/lda/si-q4")
    
        num. electrons       : 64
        spin polarization    : none
        temperature          : 0 Ha
    
        terms                : Kinetic()
                               AtomicLocal()
                               AtomicNonlocal()
                               Ewald()
                               PspCorrection()
                               

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.115828e+02     1.510912e+00
 * time: 0.43412208557128906
     1     1.082512e+01     8.539121e-01
 * time: 1.5703039169311523
     2    -1.225656e+01     9.976659e-01
 * time: 1.6737060546875
     3    -3.433019e+01     6.866808e-01
 * time: 1.7497351169586182
     4    -4.812863e+01     5.199914e-01
 * time: 1.8307180404663086
     5    -5.724812e+01     1.765181e-01
 * time: 1.9212701320648193
     6    -5.996681e+01     8.073941e-02
 * time: 1.9977951049804688
     7    -6.087872e+01     5.921618e-02
 * time: 2.050239086151123
     8    -6.125483e+01     6.185413e-02
 * time: 2.101639986038208
     9    -6.155638e+01     3.062553e-02
 * time: 2.1543490886688232
    10    -6.176771e+01     2.654657e-02
 * time: 2.2063279151916504
    11    -6.192530e+01     2.083807e-02
 * time: 2.2764620780944824
    12    -6.201218e+01     2.022007e-02
 * time: 2.3333020210266113
    13    -6.208941e+01     1.568147e-02
 * time: 2.38611102104187

In [5]:
scfres.energies

Energy breakdown (in Ha):
    Kinetic             25.7671076
    AtomicLocal         -18.8557723
    AtomicNonlocal      14.8522673
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485387 
    Xc                  -19.3336826

    total               -62.261666456746